In [1]:
from SehirParser import *

import psycopg2

import ast

In [2]:
df = pd.read_csv("datasets/fb_users2.csv", index_col="Unnamed: 0")
df.head()

,id,title,membership,first_name,middle_name,last_name,nick_name,full_name
0,532625750441045,NaN,Sehir Dersler&Hocalar,ecenur,NaN,temelli,NaN,ecenur temelli
1,388277361642482,NaN,Sehir Dersler&Hocalar,ahmet,selman,kus,NaN,ahmet selman kus
2,2063118013923420,NaN,Sehir Dersler&Hocalar,aysecan,NaN,gurbuzler,NaN,aysecan gurbuzler
3,197256327682579,NaN,Sehir Dersler&Hocalar,ask,NaN,ihya,NaN,ask ihya
4,324997921353280,NaN,Sehir Dersler&Hocalar,farhana,NaN,zakaria,NaN,farhana zakaria


In [3]:
df.groupby("membership").count()

,id,title,first_name,middle_name,last_name,nick_name,full_name
membership,,,,,,,
Sehir Dersler&Hocalar,2346,1,2341,506,2341,0,2341
Sehir Duyuru,309,1,305,61,306,0,306
Sehir Lessons&Teachers,39,0,39,10,39,0,39
Sehir Mezunlari,35,1,35,7,34,0,35
Sehir Muhendislik,25,0,25,3,25,0,25


In [4]:
# df.head(10).to_csv("datasets/fb_users_toy.csv")

In [5]:
cols = {"id":"id", "name":"full_name", "screen_name":"full_name"}
fb_sp = SehirParser('datasets/contacts.csv', "datasets/fb_users2.csv", cols)

In [6]:
fb_sp.twitter_users_count

2754

In [7]:
fb_df, sehir_matches_fb_df = fb_sp.get_sehir_matches_df()

/home/ammar/Sehir_Network_Analysis/SehirParser.py:53: UserWarning: NaN name
  warnings.warn("NaN name")
/home/ammar/Sehir_Network_Analysis/SehirParser.py:53: UserWarning: NaN name
  warnings.warn("NaN name")
/home/ammar/Sehir_Network_Analysis/SehirParser.py:53: UserWarning: NaN name
  warnings.warn("NaN name")
/home/ammar/Sehir_Network_Analysis/SehirParser.py:53: UserWarning: NaN name
  warnings.warn("NaN name")
/home/ammar/Sehir_Network_Analysis/SehirParser.py:53: UserWarning: NaN name
  warnings.warn("NaN name")


In [8]:
sehir_matches_fb_df.head()

,full_name,sehir_matches
0,tevfik melih ertuzun,"[(tevfik melih ertuzun, 100)]"
1,yavuz gozeller,"[(yavuz gozeller, 100)]"
2,hamza msrl,"[(hamza misirli, 87)]"
3,derya guven,"[(derya guven, 100)]"
4,abdullah eser,"[(abdullah eser, 100)]"


In [9]:
len(sehir_matches_fb_df)

1916

In [10]:
fb_df["sehir_matches"] = fb_df["sehir_matches"].apply(lambda x: x[0][0])
fb_df = fb_df.rename(columns={"GUID":"fb_ID"}).set_index("fb_ID")
fb_df = fb_df[["full_name","sehir_matches","membership"]]
fb_df.head()

,full_name,sehir_matches,membership
fb_ID,,,
1510343515752802,tevfik melih ertuzun,tevfik melih ertuzun,Sehir Dersler&Hocalar
10156003240154666,yavuz gozeller,yavuz gozeller,Sehir Dersler&Hocalar
380747599053506,hamza msrl,hamza misirli,Sehir Dersler&Hocalar
10212503946487716,derya guven,derya guven,Sehir Dersler&Hocalar
1975299406043917,abdullah eser,abdullah eser,Sehir Dersler&Hocalar


In [11]:
fb_df.to_csv('datasets/sehir_fb_matches2.csv', index_label="fb_ID")

In [12]:
len(fb_df)

1940

In [13]:
fb_df.groupby("sehir_matches").count().shape

(1874, 2)

## Twitter matches

In [17]:
twitter_matches_by_guid = pd.read_csv('datasets/sehir_matches.csv', index_col="GUID.1").drop("GUID", axis=1)
twitter_matches_by_guid["sehir_matches"] = twitter_matches_by_guid["sehir_matches"].apply(lambda x: ast.literal_eval(x)[0][0])
twitter_matches_by_guid = twitter_matches_by_guid.drop("twitter_name",axis=1).reset_index()\
                                .rename(columns={
                                    "cleaned_twitter_name":"twitter_name",
                                    "GUID.1":"twitter_ID"}).set_index("twitter_ID")
twitter_matches_by_guid.head()

,sehir_matches,twitter_screen_name,profile_description,followers_count,friends_count,favourites_count,statuses_count,lang,twitter_name
twitter_ID,,,,,,,,,
1032253818,cem kurtulus,cemkurtulmus,NaN,426,284,945,1727,tr,cemkurtulmus
1241505234,caner ozdemir,AMASYABELBASKAN,NaN,4338,333,65,3559,tr,cafer ozdemir
190493057,ebru gunes,EbruGundes,NaN,2019824,46,3,5214,tr,ebru gundes
145723363,sumeyye zengin,summeye,NaN,116,358,288,634,tr,sumeyye zengin
2279504925,ahmet kaya,ahmet_kayaa44,NaN,450,1949,45,46,tr,ahmet kaya


In [18]:
twitter_matches_by_guid.groupby("sehir_matches").count().shape

(1519, 8)

In [19]:
twitter_fb = twitter_matches_by_guid.reset_index().merge(fb_df.reset_index(), left_on="sehir_matches", right_on="sehir_matches")
twitter_fb.head()

,twitter_ID,sehir_matches,twitter_screen_name,profile_description,followers_count,friends_count,favourites_count,statuses_count,lang,twitter_name,fb_ID,full_name,membership
0,190493057,ebru gunes,EbruGundes,NaN,2019824,46,3,5214,tr,ebru gundes,967246670094604,ebru gunes,Sehir Dersler&Hocalar
1,150218525,ebru gunes,burcugunes,NaN,653528,126,1708,1789,tr,burcu gunes,967246670094604,ebru gunes,Sehir Dersler&Hocalar
2,2279504925,ahmet kaya,ahmet_kayaa44,NaN,450,1949,45,46,tr,ahmet kaya,10156243626489736,ahmet kaya,Sehir Duyuru
3,183210005,ahmet kaya,AhmetKaya_Fan,Cihan Kaya #AhmetKaya Sevdasıdır Kavgamız http...,139316,296,35491,33185,tr,ahmet kaya,10156243626489736,ahmet kaya,Sehir Duyuru
4,791882910,ahmet kaya,sozlerimle,Akşamlar böyle biter. Hep böyle dertli biter.,131960,33,5500,7018,tr,ahmet kaya,10156243626489736,ahmet kaya,Sehir Duyuru


In [20]:
twitter_fb.set_index("sehir_matches").loc["muhammed caki"]

,twitter_ID,twitter_screen_name,profile_description,followers_count,friends_count,favourites_count,statuses_count,lang,twitter_name,fb_ID,full_name,membership
sehir_matches,,,,,,,,,,,,
muhammed caki,849869543522656256,Muhamme74015968,Haber Gündem Spor Futbol Müzik,2,57,1,7,tr,muhammed ali,10211725757681071,muhammed cak,Sehir Dersler&Hocalar
muhammed caki,401477209,maliustun,a değil.,366,271,1251,1186,tr,muhammed ali,10211725757681071,muhammed cak,Sehir Dersler&Hocalar
muhammed caki,938059400744456192,Muhammed4166,NaN,335,2220,1068,114,tr,muhammed ali,10211725757681071,muhammed cak,Sehir Dersler&Hocalar


In [21]:
len(fb_df)

1940

In [22]:
len(twitter_fb)

720

In [23]:
twitter_fb.groupby("sehir_matches").count().shape

(434, 12)

In [24]:
twitter_fb.to_csv('datasets/twitter_fb_matches.csv', index_label="ID")